# iBKH-based Knowledge Discovery - A Case Study for Drug Repurposing Hypothesis Generation for Alzheimer's Disease

This is the implementation of Alzheimer's Disease (AD) drug repurposing based on iBKH.

The task is to dicover drugs that potentially link to AD in the iBKH.

Generally, the script contains 3 steps, including: 1) data preparation (triplets generation); 2) knowledge graph embedding learning; and 3)predicting drug entities that potentially link to AD. For convenience, we have preprocessed the raw iBKH data to generate triplet files and trained the knowledge graph embedding models and produced embedding vectors. You may also re-produce them following the steps below.

To evaluate model performance, we use the FDA approved and drugs in clinical trials (Phase I, II, III, and IV) for AD treatment as the ground truth.

All the input iBKH raw entity & relation files, the intermediate products, i.e., triplet files and embeddings, and the drug list with the ground truth labels, can be downloaded at: https://wcm.box.com/s/jbh90entaed2jotvyab8wjsrprs4i5i8

Please make sure putting the downloaded files following the structure below.

```
.
├── ...
├── Case Study-AD Drug Repurposing.ipynb
├── Data
│   ├── iBKH                                 # iBKH raw entity & relation data - INPUT
│   │   ├── Entity 
│   │   ├── Relation
│   ├── triplets                             # Extracted triplets 
│   │   ├── DDi_triplet.csv
│   │   ├── DG_triplet.csv
│   │   ├── DiG_triplet.csv
│   │   ├── GG_triplet.csv
│   │   ├── AD_triplets.csv
│   ├── dataset
│   │   ├── training_triplets.tsv            # Extracted triplets in DGL format
│   ├── DistMult                             # KG embeddings based on DistMult
│   │   ├── entities.tsv 
│   │   ├── relations.tsv
│   │   ├── iBKH_DistMult_entity.npy
│   │   ├── iBKH_DistMult_relation.npy
│   ├── ComplEx                              # KG embeddings based on ComplEx
│   │   ├── entities.tsv 
│   │   ├── relations.tsv
│   │   ├── iBKH_ComplEx_entity.npy
│   │   ├── iBKH_ComplEx_relation.npy
│   ├── TransE_l2                            # KG embeddings based on TransE
│   │   ├── entities.tsv 
│   │   ├── relations.tsv
│   │   ├── iBKH_TransE_l2.npy
│   │   ├── iBKH_TransE_l2.npy
│   ├── TransR                               # KG embeddings based on TransR
│   │   ├── entities.tsv 
│   │   ├── relations.tsv
│   │   ├── iBKH_TransR_entity.npy
│   │   ├── iBKH_TransR_relation.npy
│   ├── Drug_list                            # FDA approved and drugs in clinical trials for AD treatment
│   │   ├── drugs_list_approve_phase1234.csv
│   │   ├── drugs_list_approve_phase234.csv
│   │   ├── drugs_list_approve_phase34.csv
│   │   ├── drugs_list_approve_phase4.csv
│   │   ├── drugs_list_approve.csv
├── predict_result                           # Model performance
│   │   ├── roc_figures
│   └── ...
└── ...
```

In [ ]:
import pandas as pd
import numpy as np

import torch as th
import torch.nn.functional as fn

from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt

### Step 1:  Generate Triplet Set from iBKH 

A triplet, i.e., (h, r, t), is the basic unit for a knowledge graph. We generate triplet set from iBKH, which will be used for knowledge graph embedding learning. Of note, in this drug repurposing case study, we only use triplets among gene, disease, and drug entities in iBKH, due to that: 1) involving all information in iBKH will lead to ~50M triplets, which will cause massive computational cost; 2) underlying indication information of drugs are likely to be captured by the context inforamtion of the drug, diese, gene entities in the KG. 


In [ ]:
kg_folder = 'Data/iBKH/' # The folder is used to store the iBKH results, include Entity results and Relation results
triplet_path = 'Data/triplets/' # The folder is used to store processed results
if not os.path.exists(triplet_path):
    os.makedirs(triplet_path)

In [ ]:
# Extracting triplets between drug and disease entities

def DDi_triplets(): 
    ddi = pd.read_csv(kg_folder + 'Relation/D_Di_res.csv')

    ddi_treats = ddi[ddi['Treats'] == 1]
    ddi_treats['Relation'] = ['Treats_DDi'] * len(ddi_treats)
    ddi_treats = ddi_treats[['Drug', 'Relation', 'Disease', 'Inference_Score']]

    ddi_palliates = ddi[ddi['Palliates'] == 1]
    ddi_palliates['Relation'] = ['Palliates_DDi'] * len(ddi_palliates)
    ddi_palliates = ddi_palliates[['Drug', 'Relation', 'Disease', 'Inference_Score']]

    ddi_effect = ddi[ddi['Effect'] == 1]
    ddi_effect['Relation'] = ['Effect_DDi'] * len(ddi_effect)
    ddi_effect = ddi_effect[['Drug', 'Relation', 'Disease', 'Inference_Score']]

    ddi_associate = ddi[ddi['Associate'] == 1]
    ddi_associate['Relation'] = ['Associate_DDi'] * len(ddi_associate)
    ddi_associate = ddi_associate[['Drug', 'Relation', 'Disease', 'Inference_Score']]

    ddi_IR = ddi[ddi['Inferred_Relation'] == 1]
    ddi_IR['Relation'] = ['Inferred_Relation_DDi'] * len(ddi_IR)
    ddi_IR = ddi_IR[['Drug', 'Relation', 'Disease', 'Inference_Score']]

    ddi_SR = ddi[
        (ddi['treatment/therapy (including investigatory)'] == 1) | (ddi['inhibits cell growth (esp. cancers)'] == 1) |
        (ddi['alleviates, reduces'] == 1) | (ddi['biomarkers (of disease progression)'] == 1) |
        (ddi['prevents, suppresses'] == 1) | (ddi['role in disease pathogenesis'] == 1)]
    ddi_SR['Relation'] = ['Semantic_Relation_DDi'] * len(ddi_SR)
    ddi_SR = ddi_SR[['Drug', 'Relation', 'Disease', 'Inference_Score']]

    ddi_res = pd.concat((ddi_treats, ddi_palliates, ddi_effect, ddi_associate, ddi_IR, ddi_SR))
    ddi_res = ddi_res.rename(columns={'Drug': 'Head', 'Disease': 'Tail'})

    ddi_res.loc[ddi_res['Relation'] != 'Inferred_Relation_DDi', 'Inference_Score'] = np.nan

    ddi_res.to_csv('Data/triplets/DDi_triplet.csv', index=False)

In [ ]:
# Extracting triplets between drug and gene entities.

def DG_triplets():
    dg = pd.read_csv(kg_folder + 'Relation/D_G_res.csv')

    dg_target = dg[dg['Target'] == 1]
    dg_target['Relation'] = ['Target_DG'] * len(dg_target)
    dg_target['Inference_Score'] = [''] * len(dg_target)
    dg_target = dg_target[['Drug', 'Relation', 'Gene', 'Inference_Score']]

    dg_transporter = dg[dg['Transporter'] == 1]
    dg_transporter['Relation'] = ['Transporter_DG'] * len(dg_transporter)
    dg_transporter['Inference_Score'] = [''] * len(dg_transporter)
    dg_transporter = dg_transporter[['Drug', 'Relation', 'Gene', 'Inference_Score']]

    dg_enzyme = dg[dg['Enzyme'] == 1]
    dg_enzyme['Relation'] = ['Enzyme_DG'] * len(dg_enzyme)
    dg_enzyme['Inference_Score'] = [''] * len(dg_enzyme)
    dg_enzyme = dg_enzyme[['Drug', 'Relation', 'Gene', 'Inference_Score']]

    dg_carrier = dg[dg['Carrier'] == 1]
    dg_carrier['Relation'] = ['Carrier_DG'] * len(dg_carrier)
    dg_carrier['Inference_Score'] = [''] * len(dg_carrier)
    dg_carrier = dg_carrier[['Drug', 'Relation', 'Gene', 'Inference_Score']]

    dg_downregulates = dg[dg['Downregulates'] == 1]
    dg_downregulates['Relation'] = ['Downregulates_DG'] * len(dg_downregulates)
    dg_downregulates['Inference_Score'] = [''] * len(dg_downregulates)
    dg_downregulates = dg_downregulates[['Drug', 'Relation', 'Gene', 'Inference_Score']]

    dg_upregulates = dg[dg['Upregulates'] == 1]
    dg_upregulates['Relation'] = ['Upregulates_DG'] * len(dg_upregulates)
    dg_upregulates['Inference_Score'] = [''] * len(dg_upregulates)
    dg_upregulates = dg_downregulates[['Drug', 'Relation', 'Gene', 'Inference_Score']]

    dg_associate = dg[dg['Associate'] == 1]
    dg_associate['Relation'] = ['Associate_DG'] * len(dg_associate)
    dg_associate['Inference_Score'] = [''] * len(dg_associate)
    dg_associate = dg_associate[['Drug', 'Relation', 'Gene', 'Inference_Score']]

    dg_binds = dg[dg['Binds'] == 1]
    dg_binds['Relation'] = ['Binds_DG'] * len(dg_binds)
    dg_binds['Inference_Score'] = [''] * len(dg_binds)
    dg_binds = dg_binds[['Drug', 'Relation', 'Gene', 'Inference_Score']]

    dg_interaction = dg[dg['Interaction'] == 1]
    dg_interaction['Relation'] = ['Interaction_DG'] * len(dg_interaction)
    dg_interaction['Inference_Score'] = [''] * len(dg_interaction)
    dg_interaction = dg_interaction[['Drug', 'Relation', 'Gene', 'Inference_Score']]

    dg_SR = dg[(dg['affects expression/production (neutral)'] == 1) | (dg['agonism, activation'] == 1) |
               (dg['inhibits'] == 1) | (dg['metabolism, pharmacokinetics'] == 1) | (dg['antagonism, blocking'] == 1) |
               (dg['increases expression/production'] == 1) | (dg['binding, ligand (esp. receptors)'] == 1) |
               (dg['decreases expression/production'] == 1) | (dg['transport, channels'] == 1) |
               (dg['enzyme activity'] == 1) | (dg['physical association'] == 1)]
    dg_SR['Relation'] = ['Semantic_Relation_DG'] * len(dg_SR)
    dg_SR['Inference_Score'] = [''] * len(dg_SR)
    dg_SR = dg_SR[['Drug', 'Relation', 'Gene', 'Inference_Score']]

    dg_res = pd.concat(
        (dg_target, dg_transporter, dg_enzyme, dg_carrier, dg_downregulates, dg_upregulates, dg_associate,
         dg_binds, dg_interaction, dg_SR))
    dg_res = dg_res.rename(columns={'Drug': 'Head', 'Gene': 'Tail'})

    dg_res.to_csv('Data/triplets/DG_triplet.csv', index=False)

In [ ]:
# Extracting triplets between disease and gene entities.

def DiG_triplets():
    dig = pd.read_csv(kg_folder + 'Relation/Di_G_res.csv')

    dig_associate = dig[dig['Associate'] == 1]
    dig_associate['Relation'] = ['Associate_DiG'] * len(dig_associate)
    dig_associate = dig_associate[['Disease', 'Relation', 'Gene', 'Inference_Score']]

    dig_downregulates = dig[dig['Downregulates'] == 1]
    dig_downregulates['Relation'] = ['Downregulates_DiG'] * len(dig_downregulates)
    dig_downregulates = dig_downregulates[['Disease', 'Relation', 'Gene', 'Inference_Score']]

    dig_upregulates = dig[dig['Upregulates'] == 1]
    dig_upregulates['Relation'] = ['Upregulates_DiG'] * len(dig_upregulates)
    dig_upregulates = dig_upregulates[['Disease', 'Relation', 'Gene', 'Inference_Score']]

    dig_IR = dig[dig['Inferred_Relation'] == 1]
    dig_IR['Relation'] = ['Inferred_Relation_DiG'] * len(dig_IR)
    dig_IR = dig_IR[['Disease', 'Relation', 'Gene', 'Inference_Score']]

    dig_SR = dig[(dig['improper regulation linked to disease'] == 1) | (dig['causal mutations'] == 1) |
                 (dig['polymorphisms alter risk'] == 1) | (dig['role in pathogenesis'] == 1) |
                 (dig['possible therapeutic effect'] == 1) | (dig['biomarkers (diagnostic)'] == 1) |
                 (dig['promotes progression'] == 1) | (dig['drug targets'] == 1) | (
                         dig['overexpression in disease'] == 1) |
                 (dig['mutations affecting disease course'] == 1)]
    dig_SR['Relation'] = ['Semantic_Relation_DiG'] * len(dig_SR)
    dig_SR = dig_SR[['Disease', 'Relation', 'Gene', 'Inference_Score']]

    dig_res = pd.concat((dig_associate, dig_downregulates, dig_upregulates, dig_IR, dig_SR))
    dig_res = dig_res.rename(columns={'Disease': 'Head', 'Gene': 'Tail'})

    dig_res.loc[dig_res['Relation'] != 'Inferred_Relation_DiG', 'Inference_Score'] = np.nan

    dig_res.to_csv('Data/triplets/DiG_triplet.csv', index=False)

In [ ]:
# Extracting triplets between gene entities.

def GG_triplets():
    gg = pd.read_csv(kg_folder + 'Relation/G_G_res.csv')

    gg_covaries = gg[gg['Covaries'] == 1]
    gg_covaries['Relation'] = ['Covaries_GG'] * len(gg_covaries)
    gg_covaries['Inference_Score'] = [''] * len(gg_covaries)
    gg_covaries = gg_covaries[['Gene_1', 'Relation', 'Gene_2', 'Inference_Score']]

    gg_interacts = gg[gg['Interacts'] == 1]
    gg_interacts['Relation'] = ['Interacts_GG'] * len(gg_interacts)
    gg_interacts['Inference_Score'] = [''] * len(gg_interacts)
    gg_interacts = gg_interacts[['Gene_1', 'Relation', 'Gene_2', 'Inference_Score']]

    gg_regulates = gg[gg['Regulates'] == 1]
    gg_regulates['Relation'] = ['Regulates_GG'] * len(gg_regulates)
    gg_regulates['Inference_Score'] = [''] * len(gg_regulates)
    gg_regulates = gg_regulates[['Gene_1', 'Relation', 'Gene_2', 'Inference_Score']]

    gg_associate = gg[gg['Associate'] == 1]
    gg_associate['Relation'] = ['Associate_GG'] * len(gg_associate)
    gg_associate['Inference_Score'] = [''] * len(gg_associate)
    gg_associate = gg_associate[['Gene_1', 'Relation', 'Gene_2', 'Inference_Score']]

    gg_SR = gg[
        (gg['activates, stimulates'] == 1) | (gg['production by cell population'] == 1) | (gg['regulation'] == 1) |
        (gg['binding, ligand (esp. receptors)'] == 1) | (gg['signaling pathway'] == 1) |
        (gg['increases expression/production'] == 1) | (gg['same protein or complex'] == 1) |
        (gg['enhances response'] == 1) | (gg['affects expression/production (neutral)'] == 1) |
        (gg['physical association'] == 1) | (gg['association'] == 1) | (gg['colocalization'] == 1) |
        (gg['dephosphorylation reaction'] == 1) | (gg['cleavage reaction'] == 1) | (gg['direct interation'] == 1) |
        (gg['ADP ribosylation reaction'] == 1) | (gg['ubiquitination reaction'] == 1) |
        (gg['phosphorylation reaction'] == 1) | (gg['protein cleavage'] == 1)]
    gg_SR['Relation'] = ['Semantic_Relation_GG'] * len(gg_SR)
    gg_SR['Inference_Score'] = [''] * len(gg_SR)
    gg_SR = gg_SR[['Gene_1', 'Relation', 'Gene_2', 'Inference_Score']]

    gg_res = pd.concat((gg_covaries, gg_interacts, gg_regulates, gg_associate, gg_SR))
    gg_res = gg_res.rename(columns={'Gene_1': 'Head', 'Gene_2': 'Tail'})

    gg_res.to_csv('Data/triplets/GG_triplet.csv', index=False)

Run functions to generate corresponding triplets

In [ ]:
DDi_triplets()
DG_triplets()
DiG_triplets()
GG_triplets()

This is an AD (Alzheimer's disease) drug repurposing task, to avoid information leaking in model training, we removed all AD-related triplets from the Drug-Disease triplet set.

In [ ]:
# This function will return a csv file, which combines all triplets for interest with removing 
# AD-related drug-disease triplets

def generate_training_set():
    disease_vocab = pd.read_csv(kg_folder + 'Entity/disease_vocab.csv')
    AD_related_list = [] # Find AD related entities from the Disease vocabulary in iBKH
    for i in range(len(disease_vocab)):
        primary_id = disease_vocab.loc[i, 'primary']
        disease_name = disease_vocab.loc[i, 'name']
        disease_name = disease_name if not pd.isnull(disease_name) else ''
        if 'alzheimer' in disease_name:
            if primary_id not in AD_related_list:
                AD_related_list.append(primary_id)

    triplet_folder = 'Data/triplets/'
    DDi_triplet = pd.read_csv(triplet_folder + 'DDi_triplet.csv')
    DDi_triplet = DDi_triplet[~(DDi_triplet['Tail'].isin(AD_related_list))]
    DDi_triplet = DDi_triplet.reset_index(drop=True)
    DG_triplet = pd.read_csv(triplet_folder + 'DG_triplet.csv')
    DiG_triplet = pd.read_csv(triplet_folder + 'DiG_triplet.csv')
    GG_triplet = pd.read_csv(triplet_folder + 'GG_triplet.csv')
    DD_triplet = pd.read_csv(triplet_folder + 'DD_triplet.csv')

    triplet_set = pd.concat((DDi_triplet, DG_triplet, DiG_triplet, GG_triplet, DD_triplet))
    triplet_set = triplet_set[['Head', 'Relation', 'Tail']]

    triplet_set.to_csv(triplet_folder + 'training_triplets.csv', index=False)

generate_training_set()  

In [ ]:
# This function converts the triplet csv file to tsv format, according to DGL requirment

def generate_training_set():
    triplets_set = pd.read_csv(triplet_folder + 'training_triplets.csv')

    triples = triplets_set.values.tolist()
    train_set = np.arange(len(triples)).tolist()
    
    dataset_path = 'Data/dataset/'
    if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)

    print(len(triples), len(train_set))
    with open(dataset_path + "training_triplets.tsv", 'w+') as f:
        for idx in train_set:
            f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))
            
generate_training_set()

### Step 2:  Knowledge graph embedding based on the triplets

We directly invoke the command line toolkit provided by DGL-KE to learn the embedding of entities and relations in iBKH. Here, we use four different models to learn the entity and edge representations of iBKH, namely TransE, TransR, DistMult, and ComplEx. To use other KGE model or AWS instances please refer to DGL-KE’s <a href="https://aws-dglke.readthedocs.io/en/latest/index.html" target="_blank">Document</a>.


Of note, for convenience, we have trained the knowledge graph embedding models and produced the embedding vectors which can be found in the downloaded folder. You may also run the command below to reproduce the embedding vectors. 

##### TransE

In [ ]:
DGLBACKEND=pytorch dglke_train --dataset iBKH --data_path ./dataset --data_files training_triplets.tsv --format raw_udd_hrt --model_name TransE_l2 --batch_size 1024 --neg_sample_size 256 --hidden_dim 200 --gamma 12.0 --lr 0.1 --max_step 10000 --log_interval 100 --batch_size_eval 1000 -adv --regularization_coef 1.00E-07 --num_thread 1 --num_proc 8 --neg_sample_size_eval 1000

##### TransR

In [ ]:
DGLBACKEND=pytorch dglke_train --dataset iBKH --data_path ./dataset --data_files training_triplets.tsv --format raw_udd_hrt --model_name TransR --batch_size 1024 --neg_sample_size 256 --hidden_dim 200 --gamma 12.0 --lr 0.005 --max_step 10000 --log_interval 100 --batch_size_eval 1000 -adv --regularization_coef 1.00E-07 --num_thread 1 --num_proc 8 --neg_sample_size_eval 1000

##### DistMult

In [ ]:
DGLBACKEND=pytorch dglke_train --dataset iBKH --data_path ./dataset --data_files training_triplets.tsv --format raw_udd_hrt --model_name DistMult --batch_size 1024 --neg_sample_size 256 --hidden_dim 400 --gamma 12.0 --lr 0.005 --max_step 10000 --log_interval 100 --batch_size_eval 1000 -adv --regularization_coef 1.00E-07 --num_thread 1 --num_proc 8 --neg_sample_size_eval 1000

##### ComplEx

In [ ]:
DGLBACKEND=pytorch dglke_train --dataset iBKH --data_path ./dataset --data_files training_triplets.tsv --format raw_udd_hrt --model_name ComplEx --batch_size 1024 --neg_sample_size 256 --hidden_dim 200 --gamma 12.0 --lr 0.05 --max_step 10000 --log_interval 100 --batch_size_eval 1000 -adv --regularization_coef 1.00E-07 --num_thread 1 --num_proc 8 --neg_sample_size_eval 1000

### Step 3: Link prediction for AD drug repurposing hypothesis generation

Inspired by https://www.dgl.ai/news/2020/06/09/covid.html, we used the following algorithms to calculate the edge scores. And the edge scores indicate the strength of association between candidate entities and AD.

In [ ]:
def transE_l2(head, rel, tail, gamma=12.0):
    # Paper link: https://papers.nips.cc/paper/5071-translating-embeddings-for-modeling-multi-relational-data
    score = head + rel - tail
    
    return gamma - th.norm(score, p=2, dim=-1)

In [ ]:
def transR(head, rel, tail, proj, rel_idx, gamma=12.0):
    # Paper link: https://www.aaai.org/ocs/index.php/AAAI/AAAI15/paper/download/9571/9523
    proj = proj.reshape(-1, head.shape[1], rel.shape[0])[rel_idx]
    head_r = th.einsum('ab,bc->ac', head, proj)
    tail_r = th.einsum('b,bc->c', th.tensor(tail), proj)
    score = head_r + rel - tail_r
    
    return gamma - th.norm(score, p=1, dim=-1)

In [ ]:
def DistMult(head, rel, tail):
    # Paper link: https://arxiv.org/abs/1412.6575
    score = head * rel * tail
    
    return th.sum(score, dim=-1)

In [ ]:
def complEx(head, rel, tail, gamma=12.0):
    # Paper link: https://arxiv.org/abs/1606.06357
    real_head, img_head = th.chunk(head, 2, dim=-1)
    real_tail, img_tail = th.chunk(th.tensor(tail), 2, dim=-1)
    real_rel, img_rel = th.chunk(rel, 2, dim=-1)

    score = real_head * real_tail * real_rel \
            + img_head * img_tail * real_rel \
            + real_head * img_tail * img_rel \
            - img_head * real_tail * img_rel

    return th.sum(score, -1)

Below we define the AD_drugs_possibility_prediction function, which generates a prediction score for each drug, indicating the probablity that the drug can be used to treat AD. 

In [ ]:
folder = 'Data/'
Drug_list_folder = 'Data/Drug_list/'
kg_folder = 'Data/iBKH/'
result_folder = 'predict_result/'

In [ ]:
def AD_drugs_possibility_prediction(model_name, trial_status):
    entity_df = pd.read_table(folder + model_name + '/entities.tsv', header=None)
    entity_df = entity_df.dropna().reset_index(drop=True)
    approved_drug_df = pd.read_csv(Drug_list_folder + 'drugs_list_' + trial_status + '.csv')
    approved_drug_list = list(approved_drug_df['Drug'])

    entity_map = {}
    entity_id_map = {}
    relation_map = {}
    drug_ids = []
    drug_names = []
    disease_ids = []

    for i in range(len(entity_df)):
        entity_id = entity_df.loc[i, 0]
        entity_name = entity_df.loc[i, 1]
        entity_map[entity_name] = int(entity_id)
        entity_id_map[int(entity_id)] = entity_name
        if entity_name.replace('DrugBank:', '') in approved_drug_list:
            drug_ids.append(entity_id)
            drug_names.append(entity_name.replace('DrugBank:', ''))

    disease_vocab = pd.read_csv(kg_folder + 'Entity/disease_vocab.csv')
    AD_related_list = []
    for i in range(len(disease_vocab)):
        primary_id = disease_vocab.loc[i, 'primary']
        disease_name = disease_vocab.loc[i, 'name']
        disease_name = disease_name if not pd.isnull(disease_name) else ''
        if 'alzheimer' in disease_name:
            if primary_id not in AD_related_list:
                AD_related_list.append(primary_id)

    relation_df = pd.read_table(folder + model_name + '/relations.tsv', header=None)
    for i in range(len(relation_df)):
        relation_id = relation_df.loc[i, 0]
        relation_name = relation_df.loc[i, 1]
        relation_map[relation_name] = int(relation_id)

    for disease in AD_related_list:
        if disease in entity_map:
            disease_ids.append(entity_map[disease])

    entity_emb = np.load(folder + model_name + '/iBKH_' + model_name + '_entity.npy')
    rel_emb = np.load(folder + model_name + '/iBKH_' + model_name + '_relation.npy')
    if model_name == 'TransR':
        proj_np = np.load(folder + 'TransR/iBKH_TransRprojection.npy')
        proj_emb = th.tensor(proj_np)

    treatment = ['Treats_DDi', 'Palliates_DDi', 'Effect_DDi', 'Associate_DDi', 'Inferred_Relation_DDi',
                 'Semantic_Relation_DDi']
    treatment_rid = [relation_map[treat] for treat in treatment]

    drug_ids = th.tensor(drug_ids).long()
    disease_ids = th.tensor(disease_ids).long()
    treatment_rid = th.tensor(treatment_rid)

    drug_emb = th.tensor(entity_emb[drug_ids])
    treatment_embs = [th.tensor(rel_emb[rid]) for rid in treatment_rid]

    scores_per_disease = []
    dids = []
    for rid in range(len(treatment_embs)):
        treatment_emb = treatment_embs[rid]
        for disease_id in disease_ids:
            disease_emb = th.tensor(entity_emb[disease_id])
            if model_name == 'RotatE':
                score = fn.logsigmoid(rotatE(drug_emb, treatment_emb, disease_emb))
            elif model_name == 'ComplEx':
                score = fn.logsigmoid(complEx(drug_emb, treatment_emb, disease_emb))
            elif model_name == 'TransR':
                score = fn.logsigmoid(transR(drug_emb, treatment_emb, disease_emb, proj_emb, treatment_rid[rid]))
            elif model_name == 'TransE_l2':
                score = fn.logsigmoid(transE_l2(drug_emb, treatment_emb, disease_emb))
            elif model_name == 'DistMult':
                score = fn.logsigmoid(DistMult(drug_emb, treatment_emb, disease_emb))
            scores_per_disease.append(score)
            dids.append(drug_ids)
    scores = th.cat(scores_per_disease)
    dids = th.cat(dids)

    idx = th.flip(th.argsort(scores), dims=[0])
    scores = scores[idx].numpy()
    dids = dids[idx].numpy()

    _, unique_indices = np.unique(dids, return_index=True)
    topk_indices = np.sort(unique_indices)
    proposed_dids = dids[topk_indices]
    proposed_scores = scores[topk_indices]

    candidate_drug_rank = []
    candidate_drug_score = {}
    for i, idx in enumerate(proposed_dids):
        candidate_drug_rank.append(entity_id_map[int(idx)].replace('DrugBank:', ''))
        candidate_drug_score[entity_id_map[int(idx)].replace('DrugBank:', '')] = proposed_scores[i]

    df = pd.DataFrame(columns=['Drug', 'Score'])
    idx = 0
    for drug in candidate_drug_score:
        df.loc[idx] = [drug, candidate_drug_score[drug]]
        idx += 1

    x = np.asarray(df['Score']).reshape(-1, 1)  # returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df['Score_scaled'] = pd.DataFrame(x_scaled)
    print(df)
    df.to_csv(result_folder + "predict_result_scaled_" + model_name + "_" + trial_status + ".csv", index=False)

We propose an ensemble model, which combines the results based on TransE, TransR, DistMult, and complEx to claculate a combined score for each drug.

In [ ]:
def ensemble_model(trial_status):
    transE_res = pd.read_csv(result_folder + "predict_result_scaled_TransE_l2_" + trial_status + ".csv")['Drug'].tolist()
    transR_res = pd.read_csv(result_folder + "predict_result_scaled_TransR_" + trial_status + ".csv")['Drug'].tolist()
    complEx_res = pd.read_csv(result_folder + "predict_result_scaled_ComplEx_" + trial_status + ".csv")['Drug'].tolist()
    distMult_res = pd.read_csv(result_folder + "predict_result_scaled_DistMult_" + trial_status + ".csv")['Drug'].tolist()

    res = pd.DataFrame(columns=['Drug', 'vote_score'])
    idx = 0
    for drug in transE_res:
        vote_transE = len(transE_res) - transE_res.index(drug)
        vote_transR = len(transR_res) - transR_res.index(drug)
        vote_complEx = len(complEx_res) - complEx_res.index(drug)
        vote_distMult = len(distMult_res) - distMult_res.index(drug)
        vote_score = vote_transE + vote_transR + vote_complEx + vote_distMult
        res.loc[idx] = [drug, vote_score]
        idx += 1
    res = res.sort_values('vote_score', ascending=False)
    res = res.reset_index(drop=True)
    x = np.asarray(res['vote_score']).reshape(-1, 1)
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    res['Score_scaled'] = pd.DataFrame(x_scaled)
    print(res)
    res.to_csv(result_folder + "predict_result_scaled_ensemble_" + trial_status + ".csv", index=False)

#### Model Evaluatoin

We first downloaded all Food and Drug Administration (FDA) approved drugs and drugs in clinical trials (Phases I-IV) for AD from the DrugBank (https://go.drugbank.com/), constructing the grand truth drug list. Specifically, we obtained a total of 10 FDA-approved drugs, 30 drugs in Phase IV trials, 43 drugs in Phase III trials, 95 drugs in Phase II trials, and 47 drugs in Phase I trials for AD treatment. 

Of note, to avoid information leaking in prediction, all relations between the AD entity and any drug in the grand truth drug list in the iBKH were removed before calculating entity and relation embedding vectors. 

In [ ]:
def generate_AUC(model_name):
    figures_folder = result_folder + 'roc_figures/'
    drug_trial_list = ['approve_phase1234', 'approve_phase234', 'approve_phase34', 'approve_phase4', 'approve']
    trial_info = {'approve_phase1234': {'label': 'FDA approved+Phase I~IV', 'color': '#6a4c93'},
                  'approve_phase234': {'label': 'FDA approved+Phase II~IV', 'color': '#1982c4'},
                  'approve_phase34': {'label': 'FDA approved+Phase III,IV', 'color': '#8ac926'},
                  'approve_phase4': {'label': 'FDA approved+Phase IV', 'color': '#ffca3a'},
                  'approve': {'label': 'FDA approved', 'color': '#ff595e'}}
    plt.figure(figsize=(7, 7))

    for trial_status in drug_trial_list:
        predict_res = pd.read_csv(result_folder + "predict_result_scaled_" + model_name + "_" + trial_status + ".csv")
        candidate_df = pd.read_csv(Drug_list_folder + 'drugs_list_' + trial_status + '.csv')

        df = pd.merge(predict_res, candidate_df, on='Drug')

        label = np.array(list(df['label']))
        score = np.array(list(df['Score_scaled']))
        # score = np.array(list(df['Score']))

        fpr, tpr, thresholds = metrics.roc_curve(label, score)
        youden = tpr - fpr
        youden_J = np.max(youden)
        inds_youden_J = np.where(youden == youden_J)
        tpr_max = tpr[inds_youden_J]
        fpr_max = fpr[inds_youden_J]
        cut_off = thresholds[inds_youden_J][0]
        sensitivity = tpr_max[0]
        specificity = 1 - fpr_max[0]
        prevalence = np.where(label == 1)[0].shape[0] / label.shape[0]
        acc = (sensitivity * prevalence) + (specificity * (1 - prevalence))
        print(cut_off, sensitivity, specificity, acc)
        auc = metrics.auc(fpr, tpr)
        plt.plot(fpr, tpr, label=trial_info[trial_status]['label'] + ', AUC=' + str(round(auc, 2)),
                 color=trial_info[trial_status]['color'])

    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='gray', alpha=.8)
    plt.tick_params(labelsize=16, bottom=True, left=True)
    plt.xlabel("1 - Specificity", fontsize=12, fontweight='bold')
    plt.ylabel("Sensitivity", fontsize=12, fontweight='bold')
    plt.grid(alpha=.3)
    plt.legend(prop={'size': 12}, loc=4)
    plt.title(model_name, fontweight='bold', fontsize=18)
    # plt.show()
    plt.savefig(figures_folder + model_name + '.pdf', dpi=300)
    plt.close()

In [ ]:
# we try different strategies to build the ground truth AD drug list

drug_trial_list = [
    'approve_phase1234', # FDA Approved AD drugs + AD drugs in clinical trials Phase I, II, III and IV. 
    'approve_phase234',  # FDA Approved AD drugs + AD drugs in clinical trials Phase II, III and IV. 
    'approve_phase34',   # FDA Approved AD drugs + AD drugs in clinical trials Phase III and IV. 
    'approve_phase4',    # FDA Approved AD drugs + AD drugs in clinical trials Phase IV. 
    'approve'            # FDA Approved AD drugs only. 
]

##### Prediction & ROC curve (TransE)

In [ ]:
model_name = 'TransE_l2'
for trial_status in drug_trial_list:
    AD_drugs_possibility_prediction(model_name, trial_status)

generate_AUC(model_name)

##### Prediction & ROC curve (TransR)

In [ ]:
model_name = 'TransR'
for trial_status in drug_trial_list:
    AD_drugs_possibility_prediction(model_name, trial_status)

generate_AUC(model_name)

##### Prediction & ROC curve (DistMult)

In [ ]:
model_name = 'DistMult'
for trial_status in drug_trial_list:
    AD_drugs_possibility_prediction(model_name, trial_status)

generate_AUC(model_name)

##### Prediction & ROC curve (ComplEx)

In [ ]:
model_name = 'ComplEx'
for trial_status in drug_trial_list:
    AD_drugs_possibility_prediction(model_name, trial_status)

generate_AUC(model_name)

##### Prediction & ROC curve (Ensemble Model)

In [ ]:
for trial_status in drug_trial_list:
    ensemble_model(trial_status)

generate_AUC('ensemble')